# Imports

In [30]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/')

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
import random
from scipy import stats
import plotly.express as px
from sklearn.metrics import mean_squared_error, r2_score

import os

In [32]:
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model/model/')

In [33]:
from pipeline_config import *
from hgru_data_preprocess.preprocess_config import *

In [34]:
torch.manual_seed(1)
np.random.seed(2)
random.seed(3)

In [35]:
Year

2020

# Read Results Data

In [36]:
with open(test_predictions_path, 'rb') as f:
    prediction_dic = pickle.load(f)

with open(HRNNpath, 'rb') as f:
    raw_dataset_dict = pickle.load(f)
    
with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)

In [37]:
def create_test_dataframe(raw_dataset_dict: dict):
    test_dict = {}
    for key in raw_dataset_dict.keys():
        df = raw_dataset_dict[key][['Category', 'Date', 'Year', 'Indent', 'Inflation t+1']]
        df.dropna(inplace=True)
        df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
        target_df = df[df['Year'] > Year]
        test_dict[key] = target_df
    return test_dict

test_dict = create_test_dataframe(raw_dataset_dict)


/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_40816/1314998454.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_40816/1314998454.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_40816/1314998454.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipyker

In [38]:
def get_df_with_predictions(prediction_dic: dict, dict_of_categories_df:dict) -> dict:
    all_data_dict = {}
    for key in list(prediction_dic.keys()):
        predictions = prediction_dic[key]
        prediction_df =  pd.DataFrame(predictions.flatten().detach().numpy())
        prediction_df.rename(columns = {0: 'Prediction'}, inplace=True)
        dict_of_categories_df[key] = dict_of_categories_df[key].reset_index(drop=True)
        all_data_dict[key] = pd.concat([dict_of_categories_df[key], prediction_df], axis=1)
    return all_data_dict

all_data_test_dict=get_df_with_predictions(prediction_dic, test_dict)

In [39]:
all_categories = list(all_data_test_dict.keys())

In [40]:
def plot_results(all_data_dict, categories):
    #category_samples = random.sample(categories, 20)+['All items']
    category_samples = ['All items'] + categories
    for category in category_samples:
        category_df = all_data_dict[category]
        y_pred = category_df['Prediction'].values
        y_actual = category_df['Actual'].values
        mse = mean_squared_error(y_pred, y_actual)
        print(f'Category is: {category}')
        print(f'RMSE is: {np.sqrt(mse)}')

        fig = px.line(category_df, x="Date", y=["Actual", "Prediction"], title=f'{category} - Actual VS Prediction')
        fig.show()

#plot_results(all_data_test_dict, all_categories)

In [41]:
def avg_rmse(all_data_test_dict):
    mse_lst = []
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        mse = mean_squared_error(y_pred, y_actual)
        mse_lst.append(mse)
    
    rmse_list = list(map(np.sqrt,mse_lst))
    avg_rmse = np.mean(rmse_list)
    rmse_std = np.std(rmse_list)
    
    print(f'RMSE:  {avg_rmse}')
    print(f'MSE std:  {rmse_std}')
    print(f'interval: {[avg_rmse-rmse_std, avg_rmse+rmse_std]}')

    return avg_rmse,rmse_std

In [42]:
def total_corr(all_data_test_dict):
    corr_dict = {}
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        corr = stats.pearsonr(y_pred,y_actual)[0]
        corr_dict[key] =  corr
    
    total_corr = sum(corr_dict.values())
    #avg_corr = total_corr/len(list(all_data_test_dict.keys()))
    
    num_high_corr = 0
    for category in corr_dict:
        if corr_dict[category] >= 0.5:
            num_high_corr +=1
    
    print(f'Number of categories with High Correlation: {num_high_corr}')
    
    return total_corr

In [43]:
all_categories = list(all_data_test_dict.keys())

In [44]:
avg_rmse(all_data_test_dict)

RMSE:  1.475933460601956
MSE std:  1.1905903691628614
interval: [0.28534309143909464, 2.6665238297648175]


(1.475933460601956, 1.1905903691628614)

In [45]:
total_corr(all_data_test_dict)

Number of categories with High Correlation: 25


38.330463087211875

In [46]:
def avg_r_squared(all_data_test_dict):
    r_squared_lst = []
    low_r_squared_indent = []
    positive_r_squared_indent = []
    high_r_squared_indent = []
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        r2 = r2_score(y_actual, y_pred)
        r_squared_lst.append(r2)

        if r2<=0:
            low_r_squared_indent.append(df_predictions['Indent'].unique().item())
        
        else:
            positive_r_squared_indent.append(df_predictions['Indent'].unique().item())


        if r2>=0.5:
            high_r_squared_indent.append(df_predictions['Indent'].unique().item())

        if key =='All items':
            headline_r2 = r2
    
    avg_r2_score = np.mean(r_squared_lst)
    r2_std = np.std(r_squared_lst)
    
    print(f'Average R Squared:  {avg_r2_score}')
    print(f'R Squared std:  {r2_std}')
    print(f'interval: {[avg_r2_score-r2_std, avg_r2_score+r2_std]}')
    print('--------------------------------------------------------')
    #print(f'sorted r2 list: {sorted(r_squared_lst, reverse=True)}')
    print(f'Headline R2: {headline_r2}')
    print(f'R2 list percentiles:\n[10: {np.percentile(r_squared_lst, 10)}, 25: {np.percentile(r_squared_lst, 25)}, 50: {np.percentile(r_squared_lst, 50)}, 75: {np.percentile(r_squared_lst, 75)}, 90: {np.percentile(r_squared_lst, 90)}]')
    print(f'num of neg/zero R^2: {sum(1 for i in r_squared_lst if i<= 0)}')
    print(f'num of postive R^2: {sum(1 for i in r_squared_lst if i> 0)}')
    print(f'num of high R^2: {sum(1 for i in r_squared_lst if i>= 0.5)}')


    #return headline_r2, low_r_squared_indent, positive_r_squared_indent, high_r_squared_indent

In [47]:
avg_r_squared(all_data_test_dict)

Average R Squared:  -0.5460747552030444
R Squared std:  1.2597298494354894
interval: [-1.8058046046385337, 0.713655094232445]
--------------------------------------------------------
Headline R2: 0.02105704060627056
R2 list percentiles:
[10: -1.182989547159498, 25: -0.6333921048641031, 50: -0.30162690772048395, 75: -0.09173801001242865, 90: 0.022947368854170206]
num of neg/zero R^2: 307
num of postive R^2: 43
num of high R^2: 1


In [48]:
len(all_categories)

350

In [49]:
df_predictions = all_data_test_dict['All items']
y_pred = df_predictions['Prediction'].values
y_actual = df_predictions['Actual'].values
mse = mean_squared_error(y_pred, y_actual)
rmse = np.sqrt(mse)

print(rmse)

0.3413976060615018


In [50]:
#plot_results(all_data_test_dict, all_categories)

In [51]:
def create_rmse_table(all_data_test_dict):
    categories = []
    indents = []
    mses = []
    rmses = []
    for key in all_categories:
        categories.append(key)

        df_predictions = all_data_test_dict[key]
        indents.append(df_predictions.Indent.unique().item())
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        mse = mean_squared_error(y_pred, y_actual)

        mses.append(mse)
        rmses.append(np.sqrt(mse))
    
    series1 = pd.DataFrame(categories).reset_index(drop=True).rename(columns={0:'Category'})
    series2 = pd.DataFrame(indents).reset_index(drop=True).rename(columns={0:'Indent'})
    series3 = pd.DataFrame(mses).reset_index(drop=True).rename(columns={0:'MSE'})
    series4 = pd.DataFrame(rmses).reset_index(drop=True).rename(columns={0:'RMSE'})

    df1 = pd.concat([series1, series2], axis = 1)
    df2 = pd.concat([df1, series3], axis = 1)
    df3 = pd.concat([df2, series4], axis = 1)

    return df3


In [52]:
from pathlib import Path
def rmdir(dirc):
    dirc = Path(dirc)
    for itm in dirc.iterdir():
        if itm.is_dir():
            rmdir(itm)
            print("Deleting", itm, ".... successful.")
        else:
            itm.unlink()
    dirc.rmdir()

In [53]:
path = os.getcwd()

desired_path = path + '/rmse_per_category_hgru.csv'

rmse_per_cat = create_rmse_table(all_data_test_dict)
rmse_per_cat.to_csv(desired_path, index=False)

In [54]:
rmse_per_cat.sort_values(by = 'RMSE', ascending=True)

,Category,Indent,MSE,RMSE
84,Owners’ equivalent rent of primary residence,6.0,0.007317,0.085540
132,Shelter,3.0,0.021570,0.146868
172,Rent of shelter,4.0,0.023774,0.154187
309,"Tuition, other school fees, and childcare",5.0,0.028500,0.168821
293,Rent of primary residence,5.0,0.034903,0.186823
...,...,...,...,...
37,Airline fare,6.0,33.856816,5.818661
240,"Gasoline, unleaded regular",5.0,36.967098,6.080057
333,Energy commodities,2.0,40.304646,6.348594
125,Motor fuel,3.0,45.861988,6.772148


In [55]:
good_cats = list(rmse_per_cat.sort_values(by = 'RMSE', ascending=True).head(20).Category.unique())

In [56]:
plot_results(all_data_test_dict, good_cats)

Category is: All items
RMSE is: 0.3413976060615018


Category is: Owners’ equivalent rent of primary residence
RMSE is: 0.08554043333132702


Category is: Shelter
RMSE is: 0.14686779492175964


Category is: Rent of shelter
RMSE is: 0.1541869724588023


Category is: Tuition, other school fees, and childcare
RMSE is: 0.16882053342776854


...

Category is: Other services
RMSE is: 0.2577255464901192


Category is: Services less medical care services
RMSE is: 0.2592293879931568


Category is: All items less food, shelter, energy, and used cars and trucks
RMSE is: 0.26498709518766117


Category is: Water and sewerage maintenance
RMSE is: 0.26866035876692734


Category is: Limited service meals and snacks
RMSE is: 0.2747117224136263


In [57]:
categories_with_highest_diff = ["Energy services", 
                                "Girls’ apparel",
                                "Vehicle parts and equipment other than tires",
                                "Wine at home", 
                                "Carbonated drinks", 
                                "Propane, kerosene, and firewood", 
                                "Purchase of pets, pet supplies, accessories",
                                "Floor coverings",
                                "Hospital and related services", 
                                "Personal care",
                                "Fuel oil",
                                "Distilled spirits at home",
                                "Garbage and trash collection",
                                "Sports equipment",
                                "Other recreation services",
                                "Water and sewerage maintenance",
                                "Motor fuel",
                                "Services less energy services",
                                "Checking account and other bank services"
                                ]

In [58]:
plot_results(all_data_test_dict, categories_with_highest_diff)

Category is: All items
RMSE is: 0.3413976060615018


Category is: Energy services
RMSE is: 1.514408867594223


Category is: Girls’ apparel
RMSE is: 2.2358370164618413


Category is: Vehicle parts and equipment other than tires
RMSE is: 4.79697789297253


Category is: Wine at home
RMSE is: 0.46522907276084546


...

Category is: Other recreation services
RMSE is: 0.7085140376803668


Category is: Water and sewerage maintenance
RMSE is: 0.26866035876692734


Category is: Motor fuel
RMSE is: 6.772147940249378


Category is: Services less energy services
RMSE is: 0.1944483243025677


Category is: Checking account and other bank services
RMSE is: 4.013673712929953
